## [생성형AI][LLM] 데이터 없이 생성형 AI를 활용하여 개체명인식(NER) 분류 - 금융 도메인

https://dwin.tistory.com/148


In [ ]:
!pip install datasets evaluate seqeval
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=52c71d85022578576f0af919b168bdb65d353cd72f48e31db21dcdc59432ef50
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024

In [7]:
import json
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import time
from peft import LoraConfig, PeftModel, get_peft_model
import torch
import os
from transformers import AutoTokenizer, GemmaTokenizerFast, Gemma2ForTokenClassification
from transformers import DataCollatorForTokenClassification
import evaluate
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoModelForCausalLM, BitsAndBytesConfig

##### 1. NER 태그 데이터 전달

*   store : 매장명
*   menu : 메뉴명
*   count : 수량
*   option : 옵션


In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
os.environ["WANDB_MODE"] = "disabled"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
LABELS = ["B-store", "I-store", "B-menu", "I-menu", "B-count", "I-count"]


In [ ]:
id2label = {
    0: "O",
    1: "B-store",
    2: "I-store",
    3: "B-menu",
    4: "I-menu",
    5: "B-count",
    6: "I-count"
}
label2id = {
    "O": 0,
    "B-store": 1,
    "I-store": 2,
    "B-menu": 3,
    "I-menu": 4,
    "B-count": 5,
    "I-count": 6
}


In [ ]:
data = []
file_path = './re_label.json'
with open(file_path) as file:
    ner_data = json.load(file)
    for e in ner_data:
        tokens = e['tokens']
        ner_tags = e['ner_tags']

        sentence = ''.join(tokens)

        data.append({'sentence': sentence, 'tokens': tokens, 'ner_tags': ner_tags})


In [ ]:
pd.DataFrame(data, columns=['sentence', 'tokens', 'ner_tags']).to_csv('food_ner_dataset.csv', index=False)


In [ ]:
df = pd.DataFrame(data, columns=['sentence', 'tokens', 'ner_tags'])
df


,sentence,tokens,ner_tags
0,오프커피에서아메리카노핫으로2잔주문해주세요,"[오, 프, 커, 피, 에, 서, 아, 메, 리, 카, 노, 핫, 으, 로, 2, ...","[1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 4, 0, 0, 0, 5, ..."
1,롤키에서치즈롤케이크한조각이랑아메리카노아이스로하나씩부탁드립니다,"[롤, 키, 에, 서, 치, 즈, 롤, 케, 이, 크, 한, 조, 각, 이, 랑, ...","[1, 2, 0, 0, 3, 4, 4, 4, 4, 4, 5, 6, 6, 0, 0, ..."
2,"리스아라비카에서라떼에디카페인옵션으로두잔,라지사이즈로부탁드릴게요","[리, 스, 아, 라, 비, 카, 에, 서, 라, 떼, 에, 디, 카, 페, 인, ...","[1, 2, 2, 2, 2, 2, 0, 0, 3, 4, 0, 0, 0, 0, 0, ..."
3,"COFFEEROMAN에서플랫화이트두잔,소이라떼하나로총3잔주문할게요","[C, O, F, F, E, E, R, O, M, A, N, 에, 서, 플, 랫, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 4, ..."
4,"고더커피에서카푸치노2잔,시나몬추가로부탁드립니다","[고, 더, 커, 피, 에, 서, 카, 푸, 치, 노, 2, 잔, ,, 시, 나, ...","[1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 5, 6, 0, 0, 0, ..."
...,...,...,...
785,"보배쭈꾸미에서쭈꾸미철판볶음두개랑,밥은비빔밥으로주세요.","[보, 배, 쭈, 꾸, 미, 에, 서, 쭈, 꾸, 미, 철, 판, 볶, 음, 두, ...","[1, 2, 2, 2, 2, 0, 0, 3, 4, 4, 4, 4, 4, 4, 5, ..."
786,"해운대연탄생갈비에서생갈비2인분,된장찌개추가로주문할게요.","[해, 운, 대, 연, 탄, 생, 갈, 비, 에, 서, 생, 갈, 비, 2, 인, ...","[1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 4, 4, 5, 6, ..."
787,"캐롤커피에서아이스아메리카노한잔,핫초코한잔부탁드려요.","[캐, 롤, 커, 피, 에, 서, 아, 이, 스, 아, 메, 리, 카, 노, 한, ...","[1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 5, ..."
788,"만면희색에서칼국수한그릇,만두는3개만추가해주세요.","[만, 면, 희, 색, 에, 서, 칼, 국, 수, 한, 그, 릇, ,, 만, 두, ...","[1, 2, 2, 2, 0, 0, 3, 4, 4, 5, 6, 6, 0, 3, 4, ..."


In [ ]:
# 저장된 데이터에 대한 전처리 수행
df['ner_tags'] = df['ner_tags'].apply(lambda tags: [id2label[int(tag)] for tag in tags])


In [ ]:
df['ner_tags']


,ner_tags
0,"[B-store, I-store, I-store, I-store, O, O, B-m..."
1,"[B-store, I-store, O, O, B-menu, I-menu, I-men..."
2,"[B-store, I-store, I-store, I-store, I-store, ..."
3,"[B-store, I-store, I-store, I-store, I-store, ..."
4,"[B-store, I-store, I-store, I-store, O, O, B-m..."
...,...
785,"[B-store, I-store, I-store, I-store, I-store, ..."
786,"[B-store, I-store, I-store, I-store, I-store, ..."
787,"[B-store, I-store, I-store, I-store, O, O, B-m..."
788,"[B-store, I-store, I-store, I-store, O, O, B-m..."


In [ ]:
df.head()


,sentence,tokens,ner_tags
0,오프커피에서아메리카노핫으로2잔주문해주세요,"[오, 프, 커, 피, 에, 서, 아, 메, 리, 카, 노, 핫, 으, 로, 2, ...","[B-store, I-store, I-store, I-store, O, O, B-m..."
1,롤키에서치즈롤케이크한조각이랑아메리카노아이스로하나씩부탁드립니다,"[롤, 키, 에, 서, 치, 즈, 롤, 케, 이, 크, 한, 조, 각, 이, 랑, ...","[B-store, I-store, O, O, B-menu, I-menu, I-men..."
2,"리스아라비카에서라떼에디카페인옵션으로두잔,라지사이즈로부탁드릴게요","[리, 스, 아, 라, 비, 카, 에, 서, 라, 떼, 에, 디, 카, 페, 인, ...","[B-store, I-store, I-store, I-store, I-store, ..."
3,"COFFEEROMAN에서플랫화이트두잔,소이라떼하나로총3잔주문할게요","[C, O, F, F, E, E, R, O, M, A, N, 에, 서, 플, 랫, ...","[B-store, I-store, I-store, I-store, I-store, ..."
4,"고더커피에서카푸치노2잔,시나몬추가로부탁드립니다","[고, 더, 커, 피, 에, 서, 카, 푸, 치, 노, 2, 잔, ,, 시, 나, ...","[B-store, I-store, I-store, I-store, O, O, B-m..."


In [ ]:
df.isna().sum()


,0
sentence,0
tokens,0
ner_tags,0


##### 2. Hugging Face Datasets 형태로 변환


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['ner_tags'], axis=1), df['ner_tags'], test_size=0.2, random_state=42)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(632, 2) (158, 2) (632,) (158,)


In [ ]:
# 훈련 데이터셋 생성
train_data = {"sentence": x_train['sentence'], "tokens": x_train['tokens'], "ner_tags": y_train}
train_dataset = Dataset.from_dict(train_data)

# 테스트 데이터셋 생성
test_data = {"sentence": x_test['sentence'], "tokens": x_test['tokens'], "ner_tags": y_test}
test_dataset = Dataset.from_dict(test_data)

dataset = DatasetDict({"train": train_dataset, "test": test_dataset})


In [ ]:
print(type(dataset))		# <class 'datasets.arrow_dataset.Dataset'>
dataset.keys()


<class 'datasets.dataset_dict.DatasetDict'>


dict_keys(['train', 'test'])

In [ ]:
dataset


DatasetDict({
    train: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 632
    })
    test: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 158
    })
})

In [ ]:
label_list = list(id2label.values())
label_list


['O', 'B-store', 'I-store', 'B-menu', 'I-menu', 'B-count', 'I-count']

In [ ]:
label_list[0] == 'O'


True

In [ ]:
dataset['train'][0]


{'sentence': '짚신매운갈비찜에서매운갈비찜소자하나랑공깃밥두개추가요.',
 'tokens': ['짚',
  '신',
  '매',
  '운',
  '갈',
  '비',
  '찜',
  '에',
  '서',
  '매',
  '운',
  '갈',
  '비',
  '찜',
  '소',
  '자',
  '하',
  '나',
  '랑',
  '공',
  '깃',
  '밥',
  '두',
  '개',
  '추',
  '가',
  '요',
  '.'],
 'ner_tags': ['B-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'O',
  'O',
  'B-menu',
  'I-menu',
  'I-menu',
  'I-menu',
  'I-menu',
  'I-menu',
  'I-menu',
  'B-count',
  'I-count',
  'O',
  'B-menu',
  'I-menu',
  'I-menu',
  'B-count',
  'I-count',
  'O',
  'O',
  'O',
  'O']}

##### 3. 전처리


In [8]:
tokenizer = GemmaTokenizerFast.from_pretrained("google/gemma-2-2b-it", add_eos_token=True)


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

c:\Users\kdp\anaconda3\envs\project_06\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kdp\.cache\huggingface\hub\models--google--gemma-2-2b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens


['<bos>',
 '짚',
 '신',
 '매',
 '운',
 '갈',
 '비',
 '찜',
 '에',
 '서',
 '매',
 '운',
 '갈',
 '비',
 '찜',
 '소',
 '자',
 '하',
 '나',
 '랑',
 '공',
 '깃',
 '밥',
 '두',
 '개',
 '추',
 '가',
 '요',
 '.',
 '<eos>']

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                try:
                    if label[word_idx] == 'O':
                        label_ids.append(0)
                    else:
                        label_ids.append(int(label2id[label[word_idx]]))
                except ValueError:
                    label_ids.append(-100)
            else:

                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
tokenized_wnut = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/632 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/158 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


In [ ]:
seqeval = evaluate.load("seqeval")


In [ ]:
tokenized_wnut['train']['ner_tags']


[['B-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'O',
  'O',
  'B-menu',
  'I-menu',
  'I-menu',
  'I-menu',
  'I-menu',
  'I-menu',
  'I-menu',
  'B-count',
  'I-count',
  'O',
  'B-menu',
  'I-menu',
  'I-menu',
  'B-count',
  'I-count',
  'O',
  'O',
  'O',
  'O'],
 ['B-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'O',
  'O',
  'B-menu',
  'I-menu',
  'I-menu',
  'I-menu',
  'B-count',
  'I-count',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'I-store',
  'O',
  'O',
  'B-menu',
  'I-menu',
  'I-menu',
  'B-count',
  'I-count',
  'O',
  'O',
  'B-menu',
  'I-menu',
  'I-menu',
  'I-menu',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-store',
  'I-store',
  'O',
 

In [ ]:
tokenized_wnut['train']['labels']


[[-100,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  0,
  3,
  4,
  4,
  4,
  4,
  4,
  4,
  5,
  6,
  0,
  3,
  4,
  4,
  5,
  6,
  0,
  0,
  0,
  0,
  -100],
 [-100,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  0,
  3,
  4,
  4,
  4,
  5,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -100],
 [-100,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  0,
  3,
  4,
  4,
  5,
  6,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  -100],
 [-100,
  1,
  2,
  0,
  0,
  3,
  4,
  4,
  4,
  5,
  6,
  0,
  0,
  3,
  4,
  4,
  5,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -100],
 [-100,
  1,
  2,
  2,
  2,
  0,
  0,
  3,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  5,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -100],
 [-100, 1, 2, 2, 0, 0, 3, 4, 5, 6, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, -100],
 [-100, 1, 2, 0, 0, 3, 4, 4, 4, 5, 6, 0, 3, 4, 4, 5, 6, 0, 0, 0, 0, 0, -100],
 [-100,
  1,
  2,
  2,
  2,
  2,
  0,
  0,
  3,
  4,
  4,
  5,
  

In [ ]:
import torch.nn as nn
labels = list(id2label.values())

def compute_metrics(p):
    predictions, labels = p

    # ['O' 레이블 가중치]*0.5 -> 'O' 레이블로만 예측되는 문제 개선(클래스 불균형 문제)
    for i in range(len(predictions)):
        for j in range(len(predictions[i])):
            predictions[i][j][0] = predictions[i][j][0] * 0.5

    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    # Ensure labels are on the same device as the model's output
    labels = labels.to(model.device)
    # Ensure labels require gradients
    labels.requires_grad = True

    outputs = model(**inputs)
    logits = outputs.logits
    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(logits.view(-1, 7), labels.view(-1))
    return (loss, outputs) if return_outputs else loss


##### 4. 훈련


In [9]:
# LoRA 설정 (Gemma 모델에 맞게 조정)
lora_config = LoraConfig(
    r=16,  # 어댑터 랭크
    lora_alpha=32,  # 스케일링 파라미터
    target_modules=["q_proj", "k_proj", "v_proj"],  # Gemma 모델의 주요 프로젝션 모듈
    lora_dropout=0.1,
    bias="none"
)

In [11]:
use_model = "google/gemma-2-2b-it"
model = Gemma2ForTokenClassification.from_pretrained(
    use_model, num_labels=7, id2label=id2label, label2id=label2id,
    low_cpu_mem_usage=True, use_cache=False
    )

peft_model = get_peft_model(model, lora_config)
# 모델을 로딩할 때 low_cpu_mem_usage=True 옵션을 사용하면 CPU 메모리 사용을 최적화하고, GPU 메모리 부담을 줄일 수 있습니다.


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Gemma2ForTokenClassification were not initialized from the model checkpoint at google/gemma-2-2b-it and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Gradient checkpointing 활성화 : 중간 계산 결과를 저장하지 않고, 역전파할 때마다 다시 계산하도록 하여 메모리 사용량을 절약하는 기법

# Mixed Precision Training 활성화 : 훈련을 **16비트 부동 소수점(FP16)**로 진행할 수 있어 메모리 사용량이 절반으로 줄어들 수 있습니다. 이는 훈련 속도도 향상시킬 수 있으며, 메모리 부족 문제를 해결하는 데 도움이 됩니다.

# Gradient Accumulation을 사용하여 여러 배치에 대해 그래디언트를 누적시키고, 일정 횟수 후 한 번에 업데이트하는 방법을 사용할 수 있습니다. 이를 통해 실제 배치 크기를 증가시키지 않으면서도 모델 훈련을 안정적으로 할 수 있습니다.


In [ ]:
training_args = TrainingArguments(
    output_dir="gemma_food_ner_model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=3,
    fp16=True,
    num_train_epochs=30,
    weight_decay=0.1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,No log,4.278347,0.001635,0.004261,0.002363,0.235865
1,No log,2.537485,0.003666,0.005682,0.004457,0.358651
3,No log,0.743000,0.219289,0.306818,0.255773,0.751031
4,No log,0.458615,0.482911,0.622159,0.543762,0.860228
6,No log,0.239716,0.731250,0.830966,0.777926,0.927687
7,No log,0.214813,0.738966,0.832386,0.782899,0.937151
9,No log,0.182554,0.749373,0.849432,0.796272,0.941762
10,No log,0.169392,0.767651,0.849432,0.806473,0.943946
12,No log,0.163574,0.752179,0.857955,0.801593,0.945402
13,No log,0.159999,0.771465,0.867898,0.816845,0.950497


TrainOutput(global_step=780, training_loss=0.5616066223535783, metrics={'train_runtime': 2086.7998, 'train_samples_per_second': 9.086, 'train_steps_per_second': 0.374, 'total_flos': 7887244759871040.0, 'train_loss': 0.5616066223535783, 'epoch': 29.620253164556964})

In [ ]:
from google.colab import files
!zip -r gemma_food_ner_model.zip /content/gemma_food_ner_model/

  adding: content/gemma_food_ner_model/ (stored 0%)
  adding: content/gemma_food_ner_model/checkpoint-780/ (stored 0%)
  adding: content/gemma_food_ner_model/checkpoint-780/model-00001-of-00003.safetensors (deflated 53%)
  adding: content/gemma_food_ner_model/checkpoint-780/rng_state.pth (deflated 25%)
  adding: content/gemma_food_ner_model/checkpoint-780/tokenizer_config.json (deflated 95%)
  adding: content/gemma_food_ner_model/checkpoint-780/special_tokens_map.json (deflated 76%)
  adding: content/gemma_food_ner_model/checkpoint-780/trainer_state.json (deflated 79%)
  adding: content/gemma_food_ner_model/checkpoint-780/tokenizer.json (deflated 84%)
  adding: content/gemma_food_ner_model/checkpoint-780/model.safetensors.index.json (deflated 96%)
  adding: content/gemma_food_ner_model/checkpoint-780/model-00002-of-00003.safetensors (deflated 53%)
  adding: content/gemma_food_ner_model/checkpoint-780/tokenizer.model (deflated 51%)
  adding: content/gemma_food_ner_model/checkpoint-780/m

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# trainer.train()  # 훈련
# `use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
# => 모델 정의할때 넣었음!


In [ ]:
# 메모리 부족 문제 해결....(세미)
'''Gemma-2-2b와 같은 대형 모델을 훈련할 때는 Gradient Checkpointing, Mixed Precision, Gradient Accumulation과 같은 기법들을 통해 메모리 사용을 최적화할 수 있습니다.
모델 크기 축소를 고려한다면, DistilBERT와 같은 더 작은 모델을 사용하거나 부분 모델만 훈련하는 방식도 고려할 수 있습니다.
모델 병렬화와 프리징(freezing) 등의 방법을 통해 메모리 사용량을 줄이고 훈련을 효율적으로 할 수 있습니다.
위 방법들을 적용하여 CUDA out of memory 문제를 해결할 수 있을 것입니다.'''


'Gemma-2-2b와 같은 대형 모델을 훈련할 때는 Gradient Checkpointing, Mixed Precision, Gradient Accumulation과 같은 기법들을 통해 메모리 사용을 최적화할 수 있습니다.\n모델 크기 축소를 고려한다면, DistilBERT와 같은 더 작은 모델을 사용하거나 부분 모델만 훈련하는 방식도 고려할 수 있습니다.\n모델 병렬화와 프리징(freezing) 등의 방법을 통해 메모리 사용량을 줄이고 훈련을 효율적으로 할 수 있습니다.\n위 방법들을 적용하여 CUDA out of memory 문제를 해결할 수 있을 것입니다.'